<a href="https://colab.research.google.com/github/iamnileshmehta/Chatbot-projects/blob/main/Chatbot_using_tensorflow_and_NLTK_lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import tensorflow as tf
import nltk
from sklearn.preprocessing import LabelEncoder
nltk.download("punkt")
# Download the 'punkt_tab' data package
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
import json
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

# Load intents file
with open('/content/advisor.json') as file:
    data = json.load(file)

lemmatizer = WordNetLemmatizer()

# Data preparation
words = []
classes = []
documents = []
ignore_words = ['?', '!', '.', ',']

for intent in data['intents']:
    for pattern in intent['patterns']:
        # Tokenize words
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize and sort words
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(set(words))
classes = sorted(set(classes))


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
# Create training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(w.lower()) for w in word_patterns]
    for word in words:
        bag.append(1 if word in word_patterns else 0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# Shuffle and split data
np.random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model= Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),),activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation="softmax"))

#compile the model
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

#Train the model
model.fit(train_x,train_y,epochs=200,batch_size=5,verbose=1)

model.save("chatbot_model.h5")
print("Model trained and saved!")


Epoch 1/200


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0694 - loss: 2.9809    
Epoch 2/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1341 - loss: 3.0101 
Epoch 3/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0393 - loss: 3.0223     
Epoch 4/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1020 - loss: 2.9952     
Epoch 5/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1663 - loss: 2.8518 
Epoch 6/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1276 - loss: 2.8798     
Epoch 7/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2029 - loss: 2.8526 
Epoch 8/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1957 - loss: 2.7671 
Epoch 9/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2184 - loss: 2.7099 
Epoch 10/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4021 - loss: 2.7084 
Epoch 11/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2364 - loss: 2.6051 
Epoch 12/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

Model trained and saved!


In [7]:
from tensorflow.keras.models import load_model
model=load_model("chatbot_model.h5")

In [9]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow_vector = bow(sentence, words)
    res = model.predict(np.array([bow_vector]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]
    return return_list

def get_response(intents_list, intents_json):
    # Check if intents_list is empty
    if not intents_list:
        return "I'm sorry, I didn't understand that."  # or any default response
    tag = intents_list[0]['intent']
    for i in intents_json['intents']:
        if i['tag'] == tag:
            return np.random.choice(i['responses'])

print("Chatbot: Hi there! Type 'quit' to exit.")
while True:
    message = input("You: ")
    if message.lower() == 'quit':
        print("Chatbot: Goodbye!")
        break
    intents = predict_class(message)
    response = get_response(intents, data)
    print(f"Chatbot: {response}")


Chatbot: Hi there! Type 'quit' to exit.
You: gold
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Chatbot: Gold is a safe-haven investment. You can invest in physical gold, gold ETFs, or sovereign gold bonds (SGB).
You: ok
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Chatbot: Hello! How can I assist you with your finances today?
You: money
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Chatbot: It's okay to feel overwhelmed. Take one step at a time, create a budget, and focus on your goals. I'm here to guide you.
You: budget
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Chatbot: Budgeting helps you track expenses. Use the 50-30-20 rule: 50% for needs, 30% for wants, and 20% for savings. Would you like help creating a budget?
You: create budget
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Chatbot: Budgeting helps you track expenses. Use the 50-30-20 rule: 50% for needs, 30% for wants, and 20% for savings. Would you like help creating a budget?
You: yes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Chatbot: Namaste! How can I help you?